In [ ]:
from util.clean_data import clean_data
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# load the data
data = pd.read_csv('data/diamond.csv')

# Define target
y = data['color']

# perform one hot encoding and log transforms for carat and price
df = clean_data(data)

In [ ]:
# Check which feature is best for classification problem
def check_class_distribution(df, column_name):
    class_counts = df[column_name].value_counts()
    class_percentages = df[column_name].value_counts(normalize=True) * 100

    print(f"Class distribution for {column_name}:")
    for cls, count in class_counts.items():
        percentage = class_percentages[cls]
        print(f"Class {cls}: {count} ({percentage:.2f}%)")

check_class_distribution(df, 'color')
check_class_distribution(df, 'clarity')

In [ ]:
# Drop the old carat and price columns along with target color
df = df.drop(columns=['carat', 'price', 'color'])

# standardize the data
df = (df - df.mean()) / df.std()
X = df.values
#X.describe()

In [ ]:
max_depths = np.arange(2, 20, 1) # Tree depth range

K = 10 # Number of folds
CV = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)

In [ ]:
# Error storage variables
Error_train = np.empty((len(max_depths), K))
Error_test = np.empty((len(max_depths), K))

In [ ]:
# Cross validation and model training loop
k = 0
for train_index, test_index in CV.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    for i, depth in enumerate(max_depths):
        model = DecisionTreeClassifier(criterion='gini', max_depth=depth)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        Error_train[i, k] = np.mean(y_train_pred != y_train)
        Error_test[i, k] = np.mean(y_test_pred != y_test)

    k += 1

In [ ]:
plt.figure()
plt.boxplot(Error_test.T)
plt.xlabel("Model complexity (max tree depth)")
plt.ylabel(f"Test error across CV folds (K={K})")

plt.figure()
plt.plot(max_depths, Error_train.mean(axis=1), label='Train Error')
plt.plot(max_depths, Error_test.mean(axis=1), label='Test Error')
plt.xlabel("Model complexity (max tree depth)")
plt.ylabel(f"Error (misclassification rate, CV K={K})")
plt.legend()

plt.show()